## NOTE! This notebook will not run! The whole competition and submission processes are hosted on Kaggle, this is just a clone of the version submitted as baseline

In [ ]:
import sys
import os
import gc
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from PIL import Image
import cv2

# Deep Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder

# --- CONFIG ---
class CFG:
    MODEL_DIR = "/kaggle/input/dino-base/pytorch/baselines/1/" 
    
    # Competition Data Paths
    TEST_CSV = "/kaggle/input/csiro-biomass/test.csv"
    TEST_DIR = "/kaggle/input/csiro-biomass/test"
    TRAIN_CSV = "/kaggle/input/csiro-biomass/train.csv"
    
    # Model Architecture (Must match Training exactly)
    MODEL_NAME = "vit_base_patch14_dinov2.lvd142m"
    
    IMG_SIZE = 518
    BATCH_SIZE = 16

    TABULAR_INPUT_DIM = 25
    
    # Metadata
    CATEGORICAL_COLS = ['State', 'Species']
    NUMERICAL_COLS = ['Height_Ave_cm', 'Pre_GSHH_NDVI'] 
    
    # Targets
    TARGET_COLS = ['Dry_Total_g', 'GDM_g', 'Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g']
    AUX_COLS = ['Height_Ave_cm', 'Pre_GSHH_NDVI'] 

    # Checkpoints
    CHECKPOINTS = ["best-epoch16-w_r20.5381.ckpt", 
                   "best-epoch19-w_r20.5986.ckpt"] 

## Model Classes

In [ ]:
class BiomassModel(pl.LightningModule):
    def __init__(self, tabular_input_dim, target_dim=5):
        super().__init__()
        self.save_hyperparameters()

        # 1. Vision Backbone (DINOv2)
        self.dino = timm.create_model(CFG.MODEL_NAME, pretrained=False, num_classes=0, img_size=CFG.IMG_SIZE)
        
        # NOTE: Parameter freezing logic removed for clean inference definition
        dino_out_dim = self.dino.num_features

        # 2. Tabular Branch
        self.tabular_branch = nn.Sequential(
            nn.Linear(tabular_input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        # 3. Fusion Head
        self.head = nn.Sequential(
            nn.Linear(dino_out_dim + 64, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, target_dim)
        )

    def forward(self, x_tab, x_img):
        img_feat = self.dino(x_img)
        tab_feat = self.tabular_branch(x_tab)
        combined = torch.cat([img_feat, tab_feat], dim=1)
        return self.head(combined)

## Data Handling & Processing

In [ ]:
class InferenceDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df.reset_index(drop=True)
        self.paths = df['full_path'].values
        self.transforms = transforms
        
        # DUMMY Tabular Data: Create zero tensors of the expected size
        self.tabular_data = np.zeros((len(df), CFG.TABULAR_INPUT_DIM)).astype(np.float32)

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        path = self.paths[idx]
        try:
            image = np.array(Image.open(path).convert("RGB"))
        except:
            image = np.zeros((CFG.IMG_SIZE, CFG.IMG_SIZE, 3), dtype=np.uint8)
            
        # Resize to 518x518
        image = cv2.resize(image, (CFG.IMG_SIZE, CFG.IMG_SIZE))
        
        processed_img = image
        if self.transforms:
            processed_img = self.transforms(image=image)['image']
            
        img_tensor = processed_img
        tab_data = torch.tensor(self.tabular_data[idx], dtype=torch.float32)

        # Output both image and dummy tabular data
        return tab_data, img_tensor

def get_transforms():
    return A.Compose([
        A.Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

# --- Robust Scaler Logic (Kept for inverse transform) ---
def get_processor():
    print("Fitting Scaler on Training Data...")
    train_df = pd.read_csv(CFG.TRAIN_CSV)
    train_df['image_id'] = train_df['sample_id'].str.split('__').str[0]
    
    df_wide = train_df.pivot_table(
        index=['image_id'],
        columns='target_name',
        values='target',
        aggfunc='first'
    ).reset_index()
    
    main_scaler = RobustScaler()
    main_scaler.fit(df_wide[CFG.TARGET_COLS])
    print("Scaler fitted.")
    return main_scaler

## Main Inference Pipeline

In [ ]:
print("Starting Baseline Ensemble Inference with BiomassModel...")

# 1. Load & Prepare Test Data
test_df = pd.read_csv(CFG.TEST_CSV)
test_df['image_id'] = test_df['sample_id'].str.split('__').str[0]

try:
    sample_file = [f for f in os.listdir(CFG.TEST_DIR) if not f.startswith('.')][0]
    ext = os.path.splitext(sample_file)[1]
except:
    ext = ".png"

test_df['full_path'] = test_df['image_id'].apply(lambda x: os.path.join(CFG.TEST_DIR, f"{x}{ext}"))
df_unique = test_df[['image_id', 'full_path']].drop_duplicates(subset=['image_id']).reset_index(drop=True)

# 2. Create Dataset & Loader
test_ds = InferenceDataset(df_unique, get_transforms())
loader = DataLoader(test_ds, batch_size=CFG.BATCH_SIZE, shuffle=False)

# 3. Load Models
models = []

SPECIFIC_CHECKPOINTS = ["best-epoch19-w_r20.5986.ckpt", "best-epoch16-w_r20.5381.ckpt"] 

all_ckpt_files = list(Path(CFG.MODEL_DIR).rglob("*.ckpt"))
ckpt_files = [p for p in all_ckpt_files if p.name in SPECIFIC_CHECKPOINTS] 

print(f"Found {len(ckpt_files)} checkpoints. Loading for 2-model ensemble.")

if len(ckpt_files) != 2:
    print(f"Warning: Found {len(ckpt_files)} checkpoints, but expected 2. Check your SPECIFIC_CHECKPOINTS list.")
    
for p in ckpt_files:
    try:
        # Load the model, passing the required tabular_input_dim argument
        model = BiomassModel.load_from_checkpoint(
            str(p), 
            tabular_input_dim=CFG.TABULAR_INPUT_DIM 
        ) 
        model.eval().cuda()
        models.append(model)
        print(f"Loaded {p.name}")
    except Exception as e:
        print(f"Skipping {p.name}: {e}")

if len(models) < 2:
    raise RuntimeError(f"Could not load the required two models. Only loaded {len(models)}.")

# 4. Inference with Vanilla Ensemble
processor = get_processor()
all_preds = []

print("Running Inference...")

with torch.no_grad():
    for x_tab, x_img in tqdm(loader, desc="Predicting"):
        # x_tab is the dummy zero tensor
        x_tab, x_img = x_tab.cuda(), x_img.cuda()
        
        # Run forward pass, passing dummy x_tab and image x_img
        preds = torch.stack([m(x_tab, x_img) for m in models]).mean(dim=0).cpu().numpy()
        
        all_preds.append(preds)

# 5. Post-Processing & Submission
all_preds = np.vstack(all_preds)
all_preds = processor.inverse_transform(all_preds) 
all_preds = np.maximum(0, all_preds) 

print("📝 Writing Submission...")
pred_map = {img_id: preds for img_id, preds in zip(df_unique['image_id'].values, all_preds)}
target_idx = {name: i for i, name in enumerate(CFG.TARGET_COLS)}

final_values = []
sub_template = pd.read_csv(CFG.TEST_CSV)

for _, row in sub_template.iterrows():
    img_id = row['sample_id'].split('__')[0]
    t_name = row['target_name']
    
    if img_id in pred_map:
        val = pred_map[img_id][target_idx[t_name]]
        final_values.append(val)
    else:
        final_values.append(0.0)

sub_template['target'] = final_values
sub_template[['sample_id', 'target']].to_csv("submission.csv", index=False)
print("Done! submission.csv created successfully.")